### Creación del modelo colaborativo
Este notebook precalcula el modelo colaborativo de recomendación a través de una factorización SVD (Singular Value Decomposition). Se describe con más datelle la idea sobre el funcionamiento de este modelo en la memoria final del proyecto.Para calcular el modelo se ha hecho uso de la librería surprise.

Importamos las librerías necesarias

In [2]:
import pandas as pd
import numpy as np
import pickle
from surprise import SVD, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate

Cargamos los datos con las valoraciones de películas que han hecho los usuarios, que serán la matriz de retroalimentación para entrenar el modelo.

In [3]:
reader = Reader()
ratings = pd.read_csv('ratings_ordered.csv')
ratings.head()

,userId,rating,movieId
0,270896,5.0,110
1,270896,5.0,858
2,270896,5.0,595
3,270896,5.0,364
4,270896,5.0,4995


Convertimos los datos al formato que requiere el modelo para entrenarse.

In [4]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

Creamos el modelo colaborativo con una factorización SVD (Singular Value Decomposition) y los probamos utilizando validación cruzada para 5 particiones diferentes. Para cada una de ellas mostramos la raíz del error cuadrático medio y el error absoluto medio. 

In [6]:
collabModel = SVD()
cv=5
testResults = cross_validate(collabModel, data, measures=['rmse', 'mae'], cv=cv)
for numResult in range(cv):
    print("Partición", numResult+1)
    print("RECM (raíz del error cuadrático medio): ", testResults["test_rmse"][numResult])
    print("EAM (error absoluto medio): ", testResults["test_mae"][numResult])
    print("-------------------------------------------")

Partición 1
RECM (raíz del error cuadrático medio):  0.9103103537224058
EAM (error absoluto medio):  0.7020016815976692
-------------------------------------------
Partición 2
RECM (raíz del error cuadrático medio):  0.9123623587119938
EAM (error absoluto medio):  0.7059291024615435
-------------------------------------------
Partición 3
RECM (raíz del error cuadrático medio):  0.9244475336829978
EAM (error absoluto medio):  0.7115302666472793
-------------------------------------------
Partición 4
RECM (raíz del error cuadrático medio):  0.9081218353269808
EAM (error absoluto medio):  0.7018360070130808
-------------------------------------------
Partición 5
RECM (raíz del error cuadrático medio):  0.9102098874965066
EAM (error absoluto medio):  0.6990537737937982
-------------------------------------------


Entrenamos el modelo colaborativo con todos nuestros datos.

In [7]:
data_train = data.build_full_trainset()
collabModel.fit(data_train)

Mostramos las puntuaciones reales y predichas por el modelo para uno de los usuarios.

In [14]:
userID = 270896
moviesIDs = [110, 4995, 19, 70, 64]
for movieID in moviesIDs:
    puntReal = ratings[np.logical_and(ratings['userId'] == userID, ratings['movieId'] == movieID)]["rating"].values
    if puntReal.size > 0:
        print("Puntuación real para la película {} del usuario {}: {:.2f}".format(movieID, userID, puntReal[0]))
    else:
        print("Película aún sin valorar por el usuario")
    print("Puntuación predicha para la película {} del usuario {}: {:.2f}\n".format(movieID, userID, collabModel.predict(userID, movieID).est))

Puntuación real para la película 110 del usuario 270896: 5.00
Puntuación predicha para la película 110 del usuario 270896: 4.16

Puntuación real para la película 4995 del usuario 270896: 5.00
Puntuación predicha para la película 4995 del usuario 270896: 4.52

Puntuación real para la película 19 del usuario 270896: 2.00
Puntuación predicha para la película 19 del usuario 270896: 2.73

Película aún sin valorar por el usuario
Puntuación predicha para la película 70 del usuario 270896: 3.66

Película aún sin valorar por el usuario
Puntuación predicha para la película 64 del usuario 270896: 3.18



Finalmente, almacenamos el modelo que acabamos de computar en el fichero externo "collabModel.pkl".

In [9]:
fileTable = open("collabModel.pkl", "wb")
pickle.dump(collabModel, fileTable)
fileTable.close()